In [134]:
import numpy as np
from scipy.constants import pi, e, k, mu_0

In [135]:
# Constant terms
C_loss = 6 * pi**2 * 10e19 * 10e-3 * e  # for W in MW.s
C_fus = 17.59 * e * 1.18e-24 * 10**(2*19) * pi**2 / 2  # for P_fus in MW
C_beta = 4e2 * mu_0 * 1e19 * 1e3 * e  # beta expressed in %
C_n = 10/pi  # for n in 10^19 m^-3
C_I = 2 * pi * 1e-6 / mu_0  # for I in MA
C_SL = 0.0562  # confinement time in s

In [136]:
def nTtau(M=1, kappa=1, epsilon=1, qa=1, n_N=1, R=1, B=1, beta_N=1):
    # (n*T*Tau)^0.31
    _nTTau = C_SL * C_n**0.41 * C_I **0.96 * C_beta**0.38 * M**0.19 * kappa**0.09 * \
             epsilon**0.68 * qa**(-0.96) * n_N**0.41 * R**0.42 * B**0.73 * beta_N**(-0.38)
    return _nTTau**(1/0.31)

In [137]:
def RB1(Q=10, lambd=5, M=1, kappa=1, epsilon=1, qa=2, n_N=1, beta_N=1):
    " return R**0.42 * B**0.73 "
    # (n*T*Tau)^0.31
    _temp = C_SL * C_n**0.4 * R**0.42 * B**0.731 * C_I **0.96 * C_beta**0.38 * \
            M**0.19 * kappa**0.09 * epsilon**0.68 * qa**(-0.96) * n_N**0.41 * beta_N**(-0.38)
        
    return (C_loss/C_fus * Q/(1+Q/lambd)) / (_temp)

def RB2(P_DT=500, kappa=1, epsilon=1, qa=2, beta_N=1):
    " return R**3 * B**4 "
    return P_DT / (C_beta**(-2) * C_fus * C_I**2 * qa**(-2) * kappa * epsilon**(4) * beta_N**(2))


In [138]:
# Check consistency with ITER parameters :
# R = 6.2 m
# a = 2 m, b = 6.8 m -> kappa = 3.4, epsilon = 0.32
# Ip = 15 MA
# B = 5.3 T
# Q = 10, Pfus = 500 MW
epsilon = 0.32
kappa = 3.4
b1 = RB1(Q=10, lambd=5, kappa=kappa, epsilon=epsilon)
b2 = RB2(P_DT=500, kappa=kappa, epsilon=epsilon)

print(f'b1={b1}, b2={b2}')

# one needs to solve for (R,B) the following system:
# R**0.42 * B**0.73 = b1
# return R**3 * B**4 = b2
# multiplying by log leads to :
# 0.42 log(R) + 0.73 log(B) = log(b1)
# 3 log(R) + 4 log(B) = log(b2)

a = np.array([[0.42, 0.73], [3, 4]])
b = np.array([np.log(b1), np.log(b2)])

x = np.linalg.solve(a,b)
R = np.exp(x[0])
B = np.exp(x[1])
print(f'x={x} -> R={R}, B={B}')

b1=1067462451168.6302, b2=886836.707756209
x=[-197.62268204  151.64086556] -> R=1.4912817144679873e-86, B=7.191030747303694e+65


In [139]:
R_ITER = 6.2
B_ITER = 5.3

b1_ITER = R_ITER**0.42 * B_ITER**0.73
b2_ITER = R_ITER**3 * B_ITER**4
print(f'ITER: b1={b1_ITER}, b2={b2_ITER}')

ITER: b1=7.269857437594987, b2=188052.2555768


In [140]:
2/6.2

0.3225806451612903